# Using the Model

> This notebook explains how one can load a SysML v2 model and interact with the data

In [ ]:
from copy import deepcopy
from pathlib import Path

from ipywidgets import Dropdown

import pymbe.api as pm

In [ ]:
fixtures_folder = Path(pm.__file__).parent / "../../tests/fixtures/"

Add a dropdown selector widget to load and see all the projects

In [ ]:
models = Dropdown(
    description="Models",
    options={
        path.name.replace(".json", ""): pm.Model.load_from_file(path.resolve())
        for path in fixtures_folder.glob("*.json")
    },
)
models

Select the `Kerbal` model and assign it to `model`

In [ ]:
models.label = "Kerbal"
model = models.value

you can save the model back to a file

In [ ]:
model.save_to_file("New Kerbal.json")

and load it back...

In [ ]:
new_model = pm.Model.load_from_file("New Kerbal.json")

assert len(new_model.elements) > 0
assert len(new_model.elements) == len(model.elements)

In [ ]:
# Delete the termporarily created New Kerbal.json file
Path("New Kerbal.json").unlink()

the model can own elements (`.ownedElement`) and relationships (`.ownedRelationship`), and an element can be retrieved by name

In [ ]:
kerbal = model.ownedElement["Kerbal"]

assert kerbal.name == "Kerbal"  # the name is used to get the element
assert kerbal == kerbal._id  # the element can be directly compared to its id

kerbal

In [ ]:
kerbal(name="My Rocket")

this can be done sequentially through the chain of element ownership

In [ ]:
kerbal.ownedElement["Parts Library"].ownedElement["FL-T200 Fuel Tank"].ownedElement["Empty Mass"].ownedElement[0]["value"]

In [ ]:
assert kerbal.ownedElement["Parts Library"].ownedElement["FL-T200 Fuel Tank"].ownedElement["Empty Mass"].ownedElement[0].value == 0.125

all the model elements are stored under `model.elements` and can be retrieved by their SysML identifier

In [ ]:
for element in model.elements.values():
    if (
        element._metatype == "ReturnParameterMembership"
        and "Literal" in element.relatedElement[0]._metatype
    ):
        a_return_parameter_membership = element
        break
a_return_parameter_membership

and their properties are resolved to the appropriate element

In [ ]:
a_return_parameter_membership.relatedElement[0].value

In [ ]:
a_return_parameter_membership.relatedElement[0]

because of the nature of the data, there are multiple ways to refer to the same element, and the interactions allow for that

In [ ]:
assert a_return_parameter_membership.target[0].reverseReturnParameterMembership[0] == a_return_parameter_membership.relatedElement[0]

In [ ]:
assert a_return_parameter_membership._id == a_return_parameter_membership._data["@id"]

you can also "safely" get an attribute for an `Element`, as you would with a dictionary, using the `.get` method, the default is `None`

In [ ]:
value = a_return_parameter_membership.get("some_missing_key", "a default for something")
value

In [ ]:
assert a_return_parameter_membership.get("some_other_missing_key") is None

we can also explore all the relationships in a given element

a `through<Relationship Metatype>` is attached to the source, while a `reverse<Relationship Metatype>` is attached to the targer

In [ ]:
for element in model.elements.values():
    if element.name == "Kerbal Rocket Part":
        rocket_part = element

for subclass in rocket_part.reverseSuperclassing:
    if subclass.name == "Parachute":
        break

assert subclass.throughSuperclassing[0].name == rocket_part.name

In [ ]:
source, target = a_return_parameter_membership.relatedElement

a_return_parameter_membership.target[0].relationships
assert source.throughReturnParameterMembership[0] == target
assert target.reverseReturnParameterMembership[0] == source

we can also see all the relationships

In [ ]:
from pprint import pprint


for element_id, element in model.elements.items():
    name = element._data.get("qualifiedName") or element_id
    relationships = element.relationships

    if not relationships:
        continue
    print("\n", name)
    pprint(relationships)

In [ ]:
for element in model.elements.values():
    try:
        a_featured_value = element.get("throughFeatureValue")[0]
    except (IndexError, TypeError):
        continue
    value = a_featured_value.get("value")
    if value:
        print(f"{element} has value {value}")